In [5]:
pip install pandas google-auth google-auth-oauthlib google-auth-httplib2


Copy your API Key here

In [7]:
import pandas as pd
import re
import requests

# YouTube API Key
API_KEY = "API KEY"

# Function to extract video ID from a YouTube URL
def extract_video_id(url):
    match = re.search(r"(?:v=|\/)([0-9A-Za-z_-]{11})", url)
    return match.group(1) if match else None

# Function to fetch video details
def get_video_details(video_id):
    url = f"https://www.googleapis.com/youtube/v3/videos?part=contentDetails,snippet&id={video_id}&key={API_KEY}"
    response = requests.get(url)
    data = response.json()

    if "items" in data and len(data["items"]) > 0:
        video_data = data["items"][0]
        duration = video_data["contentDetails"]["duration"]
        channel_title = video_data["snippet"]["channelTitle"]
        return duration, channel_title
    return None, None

# Convert ISO 8601 duration format (PT#H#M#S) to seconds
def convert_duration(duration):
    match = re.match(r'PT(?:(\d+)H)?(?:(\d+)M)?(?:(\d+)S)?', duration)
    hours = int(match.group(1)) if match.group(1) else 0
    minutes = int(match.group(2)) if match.group(2) else 0
    seconds = int(match.group(3)) if match.group(3) else 0
    return hours * 3600 + minutes * 60 + seconds

# Load Excel file
file_path = "/content/Mini Drama List.xlsx"  #file path
df = pd.read_excel(file_path)

# Ensure the file has columns 'Video Name' and 'YouTube Link'
if 'YouTube Link' not in df.columns:
    raise ValueError("Excel file must have a column named 'YouTube Link'")

# Process each video link
durations = []
channel_names = []

for url in df["YouTube Link"]:
    video_id = extract_video_id(url)
    if video_id:
        duration, channel_name = get_video_details(video_id)
        if duration:
            durations.append(convert_duration(duration))
            channel_names.append(channel_name)
        else:
            durations.append(None)
            channel_names.append(None)
    else:
        durations.append(None)
        channel_names.append(None)

# Add results to DataFrame
df["Duration (Seconds)"] = durations
df["YouTube Channel"] = channel_names

# Save the updated file
output_file = "Updated Drama List.xlsx"
df.to_excel(output_file, index=False)

print(f"Updated file saved as {output_file}")


Updated file saved as Updated Drama List.xlsx
